In [11]:
import attr
from collections.abc import Sequence
import numpy as np
import numpy.typing as npt 

@attr.frozen
class Arc:
    place_index: int
    transition_index: int
    weight: int = attr.field(default=1)

@attr.mutable
class TreeNode:
    state: npt.NDArray[np.int64]
    substates: list['TreeNode'] = attr.field(factory=list)

def recurse_tree(
    transition_count: int,
    input_arcs: Sequence[Arc],
    output_arcs: Sequence[Arc],
    tree_node: TreeNode,
    all_states: list[npt.NDArray[np.int64]]
):
    for transiton_index in range(transition_count):
        def filter_by_transiton_index(arc: Arc):
            return arc.transition_index == transiton_index

        transiton_index_input_arcs = list(filter(filter_by_transiton_index, input_arcs))
        if all(map(lambda arc: tree_node.state[arc.place_index] >= arc.weight, transiton_index_input_arcs)):
            
            new_state = tree_node.state.copy()

            for arc in transiton_index_input_arcs:
                new_state[arc.place_index] -= arc.weight

            for arc in filter(filter_by_transiton_index, output_arcs):
                new_state[arc.place_index] += arc.weight

            if all(map(lambda state: not np.array_equal(new_state, state), all_states)):
                all_states.append(new_state)
                new_tree_node = TreeNode(new_state)
                tree_node.substates.append(new_tree_node)
                recurse_tree(transition_count, input_arcs, output_arcs, new_tree_node, all_states)

TreeNodeTuple = tuple[npt.NDArray[np.int64], list['TreeNodeTuple']]

def tree_node_to_tuple(tree_node: TreeNode) -> TreeNodeTuple:
    tree_node_tuple: TreeNodeTuple = (tree_node.state, [])
    for substate in tree_node.substates:
        tree_node_tuple[1].append(tree_node_to_tuple(substate))
    return tree_node_tuple

In [12]:
from pprint import pprint
transition_count = 3
root_node = TreeNode(np.array([3, 3, 0, 3]))
input_arcs: list[Arc] = [Arc(0, 0), Arc(1, 0), Arc(2, 1), Arc(2, 2, 2), Arc(3, 2)]
output_arcs = [Arc(0, 1), Arc(1, 2), Arc(2, 0), Arc(3, 1, 2)]
all_states: list[npt.NDArray[np.int64]] = [root_node.state]
recurse_tree(transition_count, input_arcs, output_arcs, root_node, all_states)
root_node_tuple = tree_node_to_tuple(root_node)
pprint(root_node_tuple)

(array([3, 3, 0, 3]),
 [(array([2, 2, 1, 3]),
   [(array([1, 1, 2, 3]),
     [(array([0, 0, 3, 3]),
       [(array([1, 0, 2, 5]),
         [(array([2, 0, 1, 7]), [(array([3, 0, 0, 9]), [])]),
          (array([1, 1, 0, 4]),
           [(array([0, 0, 1, 4]), [(array([1, 0, 0, 6]), [])])])]),
        (array([0, 1, 1, 2]), [])]),
      (array([2, 1, 1, 5]), [(array([3, 1, 0, 7]), [])]),
      (array([1, 2, 0, 2]), [])]),
    (array([3, 2, 0, 5]), [])])])


In [13]:
import scipy

input_arcs_tuples = [[arc.place_index, arc.transition_index] for arc in input_arcs]
output_arcs_tuples = [[arc.place_index, arc.transition_index] for arc in output_arcs]

input_arcs_array = np.array(input_arcs_tuples)
output_arcs_array = np.array(output_arcs_tuples)

a_minus = np.zeros((len(root_node.state), transition_count))
a_plus = np.zeros((len(root_node.state), transition_count))

a_minus[tuple(input_arcs_array.T)] = 1
a_plus[tuple(output_arcs_array.T)] = 1

a = a_plus - a_minus
a

array([[-1.,  1.,  0.],
       [-1.,  0.,  1.],
       [ 1., -1., -1.],
       [ 0.,  1., -1.]])

In [14]:
t_invariants = scipy.linalg.null_space(a)
t_invariants

array([], shape=(3, 0), dtype=float64)

In [15]:
# Vectors w (консервативність)
s_invariants = scipy.linalg.null_space(a.T)
s_invariants

array([[-5.77350269e-01],
       [ 5.77350269e-01],
       [ 1.11022302e-16],
       [ 5.77350269e-01]])